In [7]:
import librosa 
import librosa.display
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from tensorflow import keras 
from tensorflow.keras.preprocessing import image_dataset_from_directory, image 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight


# Clean Up Spectrograms 
So far I've been doing spectrograms 2 ways, and I've gotten confused between them. Lets run them both, and see if we can see the difference. Perhaps I'm missing something simple. 

In [21]:
path = Path.cwd()/'data'
answers = pd.read_csv('answers.csv')

In [9]:
def get_image_path(row):
    file_name = (row['name']+'.jpg')
    if row['label']==0:
        return images_directory/'no_whale'/file_name
    else:
        return images_directory/'whale'/file_name
    
        

In [10]:
answers['name'] = answers['clip_name'].str.split('.').str[0]
answers['clip_path']=path/'train'/answers['clip_name']
images_directory = path/'train_images'
answers['image_path']=answers.apply(get_image_path, axis=1)
images_directory = path/'train_images2'
answers['image_path2']=answers.apply(get_image_path, axis=1)
answers.head()
answers.to_csv('answers.csv')

In [35]:
def read_sound(index, df, feature=True):
    row = df.iloc[index]
    clip_path = row['clip_path']
    if feature:
        image_path = row['image_path']
    else:
        image_path = row['image_path2']
    y, sr = librosa.load(clip_path, sr=None)
    del clip_path, row
    return y, sr, image_path, feature 

def create_spectrogram(input_tuple):
    y, sr, output_file,feature = input_tuple
    n_fft=192
    
    fig = plt.figure(figsize=[1, 1])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    if feature:
        spec = librosa.feature.melspectrogram(y=y, sr=sr)
        librosa.display.specshow(librosa.power_to_db(spec, ref=np.min))
    else:
        spec = librosa.stft(y, n_fft=n_fft)
        librosa.display.specshow(librosa.power_to_db(spec, ref=np.min),n_fft=n_fft)
    plt.savefig(output_file, dpi=97, bbox_inches='tight', pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del spec, fig, ax, y, sr, output_file, input_tuple 

In [15]:
for i in tqdm(range(0, 100)):
    create_spectrogram(read_sound(i, answers, feature=False))

100%|█████████████████████████████████████████| 100/100 [00:10<00:00,  9.21it/s]


## Analysis 
There is a definite difference.The melspectrogram for feature = true, which uses the feature.melspectrogram outputs a much courser grain picture than the stft method, and it doesnt seem affected by the nfft which we're been able to control. We're able to clearly see the grain transformation from the stft. 
From here on out, we use only the image path 1 - the stft output. 

In [16]:
for i in tqdm(range(0, answers.shape[0])):
    create_spectrogram(read_sound(i, answers, feature=False))    

100%|█████████████████████████████████████| 30000/30000 [28:29<00:00, 17.55it/s]


In [36]:
for i in tqdm(range(0, answers.shape[0])):
    create_spectrogram(read_sound(i, answers, feature=True))   

100%|█████████████████████████████████████| 30000/30000 [53:42<00:00,  9.31it/s]


# Final layer Output

In Notebook 4, I made a mistake using softmax without one-hot-encoding my output. Since we are working on a binary problem, our final layer will use sigmoid and binary crossentropy, not softmax. 


# CNN & Transfer Learning 
We're tried various CNN's, and we've tried transferlearning using VGG19, but I've mixed up which spectrograms went where 

# Decided Once
Class weights should be assigned once - right below   
Metrics  
Early stopping  

# Next Notebook